In [1]:
from keys import *
import json
import pandas as pd
import numpy as np
from json.decoder import JSONDecodeError
from pandas.io.json import json_normalize
import os


In [24]:
def read_csvs(directory, df_combined):
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            csv_path = os.path.join(directory, filename)
            df_file = pd.read_csv(csv_path)
            df_combined = pd.concat([df_combined, df_file])
        else:
            continue
    return df_combined

def expand_multi_level_cols(df_combined):
    types       = ["creator", "location", "profile", "category" ]

    n           = len(df_combined)
    df_out      = pd.DataFrame()

    for type_i in types:
        df          = df_combined[type_i]
        if type_i == 'photo':
            df_loop = pd.DataFrame(columns=photo_keys)

        elif type_i == 'creator':
            df_loop = pd.DataFrame(columns=creator_keys)

        elif type_i == 'location':
            df_loop = pd.DataFrame(columns=location_keys)

        elif type_i == 'profile':
            df_loop = pd.DataFrame(columns=profile_keys)

        elif type_i == 'category':
            df_loop = pd.DataFrame(columns=category_keys)

        for i in range(0, n):
            try:
                item       = df[i]
                dic        = json.loads(item)
                x          = pd.json_normalize(dic)
                df_loop    = df_loop.append(x, ignore_index=True)
            except:
                continue
        df_out = pd.concat([df_out, df_loop], axis=1)
        return df_out
    

In [46]:

def load_data(df_type, fname):
    # This is first file from june_2021 to start dataframe (removed from june 21 folder)
    df           = pd.read_csv('./data/Kickstarter.csv')
    df_expand    = expand_multi_level_cols(df_combined)
    df_final     = df.drop(columns=["creator", "location", "profile", "category"])
    df_final     = pd.concat([df_final, df_expand], axis=1)
    idx_1        = df_final['state'] == 'failed'
    idx_2        = df_final['state'] == 'successful'
    df_final     = df_final[idx_1 | idx_2]
    pkl_file     = 'df_' + df_type + '.pkl'
    df_final.to_pickle(pkl_file)
    return df_final

# just one df takes some time to process, removing the rest for now
# df_combined  = read_csvs(d_folder_1, df_combined)
# df_combined  = read_csvs(d_f lder_2, df_combined)


In [47]:
train_fname = './data/Kickstarter.csv'
test_fname  = './data/Kickstarter001.csv'
df_train = load_data('train', train_fname)
df_test  = load_data('test', test_fname)

In [23]:
df_combined.shape, df_expand.shape, df_final.shape

((3662, 39), (3650, 14), (3662, 49))

In [21]:
df_final.head()

,backers_count,blurb,converted_pledged_amount,country,country_displayable_name,created_at,currency,currency_symbol,currency_trailing_code,current_currency,...,is_email_verified,chosen_currency,is_superbacker,avatar,urls,avatar.thumb,avatar.small,avatar.medium,urls.web.user,urls.api.user
0,65,A super-cheeky strategy card game that's full ...,2856,AU,Australia,1621558851,AUD,$,True,USD,...,None,None,None,NaN,NaN,https://ksr-ugc.imgix.net/assets/029/095/326/8...,https://ksr-ugc.imgix.net/assets/029/095/326/8...,https://ksr-ugc.imgix.net/assets/029/095/326/8...,https://www.kickstarter.com/profile/jofgames,https://api.kickstarter.com/v1/users/146087887...
1,146,Help us finish a feature film about two strang...,10120,US,the United States,1327602001,USD,$,True,USD,...,None,None,None,NaN,NaN,https://ksr-ugc.imgix.net/assets/005/910/605/a...,https://ksr-ugc.imgix.net/assets/005/910/605/a...,https://ksr-ugc.imgix.net/assets/005/910/605/a...,https://www.kickstarter.com/profile/1939666693,https://api.kickstarter.com/v1/users/193966669...
2,41,Considered playfully mischievous and outlaws i...,3785,US,the United States,1447448673,USD,$,True,USD,...,None,None,None,NaN,NaN,https://ksr-ugc.imgix.net/assets/007/087/961/6...,https://ksr-ugc.imgix.net/assets/007/087/961/6...,https://ksr-ugc.imgix.net/assets/007/087/961/6...,https://www.kickstarter.com/profile/roguecodan...,https://api.kickstarter.com/v1/users/101884698...
3,466,A fantastical love story about two New Yorkers...,21634,US,the United States,1359739327,USD,$,True,USD,...,None,None,None,NaN,NaN,https://ksr-ugc.imgix.net/assets/005/987/612/0...,https://ksr-ugc.imgix.net/assets/005/987/612/0...,https://ksr-ugc.imgix.net/assets/005/987/612/0...,https://www.kickstarter.com/profile/488675737,https://api.kickstarter.com/v1/users/488675737...
4,70,A story about a girl named Jane who is learnin...,3081,US,the United States,1330026300,USD,$,True,USD,...,None,None,None,NaN,NaN,https://ksr-ugc.imgix.net/assets/005/925/157/3...,https://ksr-ugc.imgix.net/assets/005/925/157/3...,https://ksr-ugc.imgix.net/assets/005/925/157/3...,https://www.kickstarter.com/profile/shakyballoon,https://api.kickstarter.com/v1/users/220105304...


In [26]:
df_final.to_excel('df_final.xlsx')

In [28]:
df = df_final.copy()

In [39]:
idx_1 = df['state'] == 'failed'
idx_2 = df['state'] == 'successful'

df = df[idx_1 | idx_2]

In [40]:
df_final.shape, df.shape

((3662, 49), (2536, 49))